In [1]:
import os
import json
import gzip
import pandas as pd

In [2]:
path = "C:/Users/schami/Documents/FSDL/All_Amazon_Review.json/All_Amazon_Review.json"
path = "C:/Users/schami/Documents/FSDL/Luxury_Beauty_5.json/Luxury_Beauty_5.json"
# f = open(path)
# file = json.load(f)

In [3]:
import pandas as pd
import gzip

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF('C:/Users/schami/Downloads/Luxury_Beauty_5.json.gz')

In [4]:
df

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image
0,5.0,True,"01 5, 2018",A2HOI48JK8838M,B00004U9V2,{'Size:': ' 0.9 oz.'},DB,This handcream has a beautiful fragrance. It d...,Beautiful Fragrance,1515110400,NaN,NaN
1,5.0,True,"04 5, 2017",A1YIPEY7HX73S7,B00004U9V2,{'Size:': ' 3.5 oz.'},Ajaey,"wonderful hand lotion, for seriously dry skin,...",wonderful hand lotion,1491350400,NaN,NaN
2,5.0,True,"03 27, 2017",A2QCGHIJ2TCLVP,B00004U9V2,{'Size:': ' 250 g'},D. Jones,"Best hand cream around. Silky, thick, soaks i...",Best hand cream around,1490572800,NaN,NaN
3,5.0,True,"03 20, 2017",A2R4UNHFJBA6PY,B00004U9V2,{'Size:': ' 3.5 oz.'},Amazon Customer,Thanks!!,Five Stars,1489968000,NaN,NaN
4,5.0,True,"02 28, 2017",A2QCGHIJ2TCLVP,B00004U9V2,{'Size:': ' 0.9 oz.'},D. Jones,Great hand lotion. Soaks right in and leaves ...,Great hand lotion!,1488240000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
34273,4.0,False,"09 3, 2017",A2CF66KIQ3RKX3,B01GOZ61O8,NaN,Vivian Deliz,I like to use moisturizers and sunscreens that...,Works great as a moisturizer and sunscreen,1504396800,NaN,NaN
34274,4.0,False,"09 3, 2017",A1LKOIZXPQ9VG0,B01GOZ61O8,NaN,Elisa 20,I wouldn't be able to afford this if not asked...,Nice skin care product and sunscreen if you do...,1504396800,NaN,NaN
34275,1.0,True,"08 25, 2017",AV2RWORXTFRJU,B01H353HUY,NaN,Gapeachmama,Did nothing,One Star,1503619200,NaN,NaN
34276,5.0,False,"07 8, 2017",A22S7D0LP8GRDH,B01H353HUY,NaN,Jacob and Kiki Hantla,I love the Oribe bright blonde radiance spray....,No more brass!,1499472000,NaN,NaN


In [5]:
# Keep the product ID and the comments 
dfs = df[["reviewText", "asin"]]
dfs

,reviewText,asin
0,This handcream has a beautiful fragrance. It d...,B00004U9V2
1,"wonderful hand lotion, for seriously dry skin,...",B00004U9V2
2,"Best hand cream around. Silky, thick, soaks i...",B00004U9V2
3,Thanks!!,B00004U9V2
4,Great hand lotion. Soaks right in and leaves ...,B00004U9V2
...,...,...
34273,I like to use moisturizers and sunscreens that...,B01GOZ61O8
34274,I wouldn't be able to afford this if not asked...,B01GOZ61O8
34275,Did nothing,B01H353HUY
34276,I love the Oribe bright blonde radiance spray....,B01H353HUY


In [6]:
# drop dupplicates based on review and ID
newdf = dfs.drop_duplicates(
  subset = ['reviewText', 'asin'],
  keep = 'last').reset_index(drop = True)
newdf

,reviewText,asin
0,This handcream has a beautiful fragrance. It d...,B00004U9V2
1,"wonderful hand lotion, for seriously dry skin,...",B00004U9V2
2,"Best hand cream around. Silky, thick, soaks i...",B00004U9V2
3,Thanks!!,B00004U9V2
4,Great hand lotion. Soaks right in and leaves ...,B00004U9V2
...,...,...
29855,I like to use moisturizers and sunscreens that...,B01GOZ61O8
29856,I wouldn't be able to afford this if not asked...,B01GOZ61O8
29857,Did nothing,B01H353HUY
29858,I love the Oribe bright blonde radiance spray....,B01H353HUY


In [7]:
newdf['asin'].nunique()

1581

In [8]:
newdf = newdf.dropna()

In [9]:
attributes = ["top","long-lasting", "best", "innovative", "organic","natural","GMO-free",
             "certified","Fire-retardant", "smell", "cheap", "reliable", "safe", "good", "excellent",
             "increase", "decrease", "original","rated"]
def check_attribute(text):
    has_attribute = any(att in text.lower() for att in attributes)
    return(has_attribute)
#     if type(text) == str:
#         return 1
#     return(0)

# print(has_attribute)


In [10]:
new = newdf.assign(attribute=newdf['reviewText'].apply(check_attribute))
new

,reviewText,asin,attribute
0,This handcream has a beautiful fragrance. It d...,B00004U9V2,False
1,"wonderful hand lotion, for seriously dry skin,...",B00004U9V2,False
2,"Best hand cream around. Silky, thick, soaks i...",B00004U9V2,True
3,Thanks!!,B00004U9V2,False
4,Great hand lotion. Soaks right in and leaves ...,B00004U9V2,False
...,...,...,...
29855,I like to use moisturizers and sunscreens that...,B01GOZ61O8,False
29856,I wouldn't be able to afford this if not asked...,B01GOZ61O8,True
29857,Did nothing,B01H353HUY,False
29858,I love the Oribe bright blonde radiance spray....,B01H353HUY,False


In [11]:
check = new[new['attribute'] == True].reset_index(drop = True)
check

,reviewText,asin,attribute
0,"Best hand cream around. Silky, thick, soaks i...",B00004U9V2,True
1,Best hand cream ever.,B00004U9V2,True
2,"BEST hand cream ever. One at work, one in my ...",B00004U9V2,True
3,"The Best Hand Cream Ever, I love this product....",B00004U9V2,True
4,The best hand cream ever. I'm constantly wash...,B00004U9V2,True
...,...,...,...
14043,"I like this cream, but I'm more inclined to us...",B01G2LF2VW,True
14044,"For context, I'm a 34-year-old woman with dry,...",B01G8UTFLA,True
14045,I'm a 46-year old woman with dry skin. I moist...,B01G8UTFLA,True
14046,This product doesn't really say what it is. Th...,B01GOZ61O8,True


In [12]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")
classifier("I loved Star Wars so much!")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'POSITIVE', 'score': 0.999840259552002}]

In [13]:
def get_label(text):
    res = classifier(text)
    return(res[0].get("label"))

In [14]:
def get_score(text):
    res = classifier(text)
    return(res[0].get("score"))

In [15]:
classifier(check['reviewText'].iloc[0])

[{'label': 'POSITIVE', 'score': 0.9989521503448486}]

In [16]:
    checkL = check.assign(lenght=check['reviewText'].apply(len))

In [17]:
type(check['reviewText'].iloc[0])

str

In [18]:
check = check.reset_index(drop = True)
check

,reviewText,asin,attribute
0,"Best hand cream around. Silky, thick, soaks i...",B00004U9V2,True
1,Best hand cream ever.,B00004U9V2,True
2,"BEST hand cream ever. One at work, one in my ...",B00004U9V2,True
3,"The Best Hand Cream Ever, I love this product....",B00004U9V2,True
4,The best hand cream ever. I'm constantly wash...,B00004U9V2,True
...,...,...,...
14043,"I like this cream, but I'm more inclined to us...",B01G2LF2VW,True
14044,"For context, I'm a 34-year-old woman with dry,...",B01G8UTFLA,True
14045,I'm a 46-year old woman with dry skin. I moist...,B01G8UTFLA,True
14046,This product doesn't really say what it is. Th...,B01GOZ61O8,True


In [19]:
res = classifier(check['reviewText'].iloc[134])
res

[{'label': 'NEGATIVE', 'score': 0.9996131062507629}]

In [23]:
data = []
for i in range(len(checkL)):
    try:
        res = classifier(checkL['reviewText'].iloc[i])
#     if i % 10 == 0:
        data.append(res)
    except:
        print("Error : ", i)

Error :  104
Error :  139
Error :  165
Error :  166
Error :  204
Error :  342
Error :  359
Error :  361
Error :  369
Error :  371
Error :  375
Error :  396
Error :  423
Error :  520
Error :  521
Error :  522
Error :  533
Error :  621
Error :  623
Error :  635
Error :  644
Error :  664
Error :  665
Error :  849
Error :  866
Error :  928
Error :  986
Error :  1015
Error :  1065
Error :  1106
Error :  1243
Error :  1248
Error :  1249
Error :  1260
Error :  1261
Error :  1264
Error :  1266
Error :  1278
Error :  1337
Error :  1344
Error :  1347
Error :  1449
Error :  1538
Error :  1648
Error :  1656
Error :  1671
Error :  1774
Error :  1822
Error :  1831
Error :  1902
Error :  1907
Error :  2022
Error :  2024
Error :  2051
Error :  2087
Error :  2091
Error :  2095
Error :  2116
Error :  2128
Error :  2186
Error :  2191
Error :  2204
Error :  2207
Error :  2228
Error :  2357
Error :  2370
Error :  2390
Error :  2403
Error :  2421
Error :  2441
Error :  2531
Error :  2544
Error :  2559
Error

In [25]:
checkL["reviewText"].iloc[1040]

"I received a tester.\nOne thing that I noted was that there are artificial colors in this mask when it is pretty much just white anyway.  It seems like those are just unnecessary to me.  There's also some fragrance components that could cause some people problems, though the scent itself is just very light and not at all strong.  The clay is Kaolin.\n\nIt comes out as a thick white lotion that I spread over my face.  I received a tester and there were no instructions.  I used it like I would any other mask, applying it to my face and letting it dry before rinsing it well.\n\nI picked this out to try because my skin is normally dry, but during big temperature changes like we have in the South during the spring, it gets very, very oily, and that's been a problem lately.  This has some menthol it in that felt slightly cooling on my skin.  I read while it dried and found it pleasant to have on.  It rinsed off easily.  I felt like it took every bit of oil, dirt, sweat- everything, out of m

In [51]:
    checkL = check.assign(label=check['reviewText'].apply(get_label))
# checkLS = checkL.assign(score=checkL['reviewText'].apply(get_score))
# checkLS

RuntimeError: The size of tensor a (1327) must match the size of tensor b (512) at non-singleton dimension 1

In [33]:
omar = "Love it, but a touch heavy to lug around"
classifier(omar)

[{'label': 'POSITIVE', 'score': 0.5217623710632324}]

In [34]:
test2 ="great price, great cable"
classifier(test2)

[{'label': 'POSITIVE', 'score': 0.9990334510803223}]

In [79]:
!pip install transformers

     ---------------------------------------- 4.9/4.9 MB 5.5 MB/s eta 0:00:00
  Using cached tokenizers-0.12.1-cp38-cp38-win_amd64.whl (3.3 MB)
     -------------------------------------- 120.7/120.7 kB 3.5 MB/s eta 0:00:00


In [81]:
!pip install torch torchvision torchaudio

     -------------------------------------- 161.9/161.9 MB 4.1 MB/s eta 0:00:00
     ---------------------------------------- 1.1/1.1 MB 5.6 MB/s eta 0:00:00
     -------------------------------------- 969.5/969.5 kB 5.1 MB/s eta 0:00:00


In [83]:
pip list

Package                        Version
------------------------------ -----------
appdirs                        1.4.4
argon2-cffi                    21.3.0
argon2-cffi-bindings           21.2.0
asgiref                        3.5.2
asttokens                      2.0.8
async-timeout                  4.0.2
attr                           0.3.1
attrs                          22.1.0
azure-core                     1.25.1
azure-storage-blob             12.13.1
backcall                       0.2.0
beautifulsoup4                 4.11.1
bleach                         5.0.1
boto                           2.49.0
boto3                          1.16.63
botocore                       1.19.63
boxing                         0.1.4
cachetools                     4.2.4
certifi                        2022.6.15
cffi                           1.15.1
charset-normalizer             2.0.12
click                          8.1.3
colorama                       0.4.5
coreapi                        2.3.3
coreschema  